In [1]:
from run_train import create_model_and_diffusion
from utils.step_sample import create_named_schedule_sampler
from train import TrainLoop
from utils.data import load_data_text
from tokenizer import load_tokenizer, load_model_emb
from sampling import sampling

from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast, set_seed
import json, torch, os
from utils import dist_util
from functools import partial
import pickle
import random
from datetime import datetime

2024-02-27 07:32:23.576132: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 07:32:23.608384: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 07:32:23.608426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 07:32:23.609710: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 07:32:23.615990: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dist_util.clear_cache()
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
lr=0.0001
batch_size=20
microbatch=5
epochs=30_000
eval_interval=100
ema_rate='0.9999' 
schedule_sampler='uniform'
diffusion_steps=2000
noise_schedule='sqrt'
vocab='custom'
use_plm_init='no' # embedding in transformer
vocab_size=0
config_name='bert-base-uncased'
seq_len=128
hidden_t_dim=128
hidden_dim=128
dropout=0.1
seed=102
weight_decay=0.0
predict_xstart=True
rescale_timesteps=True
emb_scale_factor=1.0

In [4]:
regular_data_dir='data'
# data_player_dir='data/with_player'

# set the data directory
data_dir=regular_data_dir

In [5]:
set_seed(seed)

In [6]:
tokenizer = load_tokenizer('shakespeare_plays', config_name)

In [7]:
model_weight, tokenizer = load_model_emb(hidden_dim, tokenizer)

In [8]:
model_weight

Embedding(30267, 128)

In [9]:
## very very important to set this!!!!!
vocab_size = tokenizer.vocab_size
vocab_size

30267

In [10]:
data = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        model_emb=model_weight # use model's weights as init
    )

val = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        split='valid',
        model_emb=model_weight # use model's weights as init
    )

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading form the TRAIN set...
### Data samples...
 ['o hell! what have we here? a carrion death, within whose empty eye there is a written scroll!', 'and his disciples only envy at, ye blew the fire that burns ye now have at ye! enter king,'] ["i'll read the writing. all that glitters is not gold, often have you heard that told", 'frowning on them, takes his seat']
RAM used: 866.29 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 48627
})
RAM used: 885.77 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/48627 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 48627
})
### tokenized_datasets...example [2, 36, 1299, 5, 163, 149, 132, 236, 21, 22, 7134, 431, 9, 905, 568, 3065, 755, 209, 120, 22, 4179, 7421, 5, 3]
RAM used: 930.33 MB


merge and mask:   0%|          | 0/48627 [00:00<?, ? examples/s]

RAM used: 967.66 MB


...

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 12147
})
### tokenized_datasets...example [2, 3885, 120, 104, 519, 9, 2545, 8, 40, 477, 9, 22, 210, 253, 1232, 9839, 186, 4042, 11, 3]
RAM used: 1034.62 MB


merge and mask:   0%|          | 0/12147 [00:00<?, ? examples/s]

RAM used: 1044.69 MB


padding:   0%|          | 0/12147 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 12147
}) padded dataset
RAM used: 1065.45 MB
RAM used: 1065.45 MB


In [12]:
model, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        diffusion_steps,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

model.to(dist_util.dev())

TransformerNetModel(
  (word_embedding): Embedding(30267, 128)
  (lm_head): Linear(in_features=128, out_features=30267, bias=True)
  (time_embed): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): SiLU()
    (2): Linear(in_features=512, out_features=768, bias=True)
  )
  (input_up_proj): Sequential(
    (0): Linear(in_features=128, out_features=768, bias=True)
    (1): Tanh()
    (2): Linear(in_features=768, out_features=768, bias=True)
  )
  (input_transformers): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768,

In [13]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

91192379

In [15]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

TrainLoop(
        model=model,
        diffusion=diffusion,
        data=data,
        batch_size=batch_size,
        microbatch=microbatch,
        lr=lr,
        ema_rate=ema_rate,
        schedule_sampler=schedule_sampler,
        weight_decay=weight_decay,
        epochs=epochs,
        eval_data=val,
        eval_interval=eval_interval,
        warm_up_steps=500,
        use_llrd=True,
        llrd_rate=0.999
    ).run_loop()



======== Using Layer-wise Learning Rate Decay with AdamW ========


name: word_embedding.weight, lr: 0.0001
name: lm_head.bias, lr: 0.0001
name: time_embed.0.weight, lr: 0.0001
name: time_embed.0.bias, lr: 0.0001
name: time_embed.2.weight, lr: 0.0001
name: time_embed.2.bias, lr: 0.0001
name: input_up_proj.0.weight, lr: 0.0001
name: input_up_proj.0.bias, lr: 0.0001
name: input_up_proj.2.weight, lr: 0.0001
name: input_up_proj.2.bias, lr: 0.0001
name: input_transformers.layer.0.attention.self.query.weight, lr: 0.0001001001001001001
name: input_transformers.layer.0.attention.self.query.bias, lr: 0.0001001001001001001
name: input_transformers.layer.0.attention.self.key.weight, lr: 0.0001001001001001001
name: input_transformers.layer.0.attention.self.key.bias, lr: 0.0001001001001001001
name: input_transformers.layer.0.attention.self.value.weight, lr: 0.0001001001001001001
name: input_transformers.layer.0.attention.self.value.bias, lr: 0.0001001001001001001
name: input_transformers.layer.0.

RuntimeError: DataLoader worker (pid(s) 5639, 5703, 5767, 5831) exited unexpectedly

In [ ]:
# param_names = []
# for i, (name, param) in enumerate(model.named_parameters()):
#     param_names.append(name)
#     print(f'{i}: {name} {param.requires_grad}')

In [ ]:
# dt = datetime.now().strftime("%m%d")
# best_model_fp = f'models/0221/model_best_epoch_20600_min_val_loss_0.028699999675154686.pkl'
# with open(best_model_fp, 'rb') as handle:
#     best_model = pickle.load(handle)

# Generating sequences

In [ ]:
_, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        2000,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

In [ ]:
word_lst_source, word_lst_recover, word_lst_ref = sampling(model, 
                                                           diffusion, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128)

Generating 20 sentences takes 5 minutes

In [ ]:
word_lst_source

In [ ]:
word_lst_recover

In [ ]:
word_lst_ref